In [1]:
from pfd_toolkit import load_reports, LLM, Screener, Extractor
from pydantic import BaseModel, Field

In [2]:
reports = load_reports(end_date="2025-11-25",
                       refresh=True)

print(f"In total, there were {len(reports)} PFD reports published between July 2013 and 25th November 2025.")

In total, there were 6043 PFD reports published between July 2013 and 25th November 2025.


In [3]:
from dotenv import load_dotenv
import os

# Load OpenAI API key from local environment
load_dotenv("api.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

# Initialise LLM client
llm_client = LLM(api_key=openai_api_key, 
                 max_workers=30, 
                 model="gpt-4.1-mini",
                 seed=123, 
                 temperature=0, 
                 timeout=60)

In [5]:
# Set up Screener
search_query = "Suicide deaths."

second_search_query = """
Deaths where the coroner raised concerns related to the insufficient involvement of the deceased's 'significant other(s)' (e.g. the deceased's family, spouse, etc.).
"""

screener = Screener(llm=llm_client, reports=reports)

suicide_reports = screener.screen_reports(search_query=search_query)
suicide_sig_reports = screener.screen_reports(reports = suicide_reports, search_query=second_search_query)

Sending requests to the LLM: 100%|██████████| 1405/1405 [00:27<00:00, 50.99it/s]


In [6]:
suicide_sig_reports.to_csv('../data/suicide_sig_reports.csv')

In [9]:
# Discover themes

extractor = Extractor(llm=llm_client,
                      reports=suicide_sig_reports)

extra_instructions = """
All themes should be sub-themes within the broader theme of the lack of sufficient role of the 'significant other' (i.e. the family, spouse, etc., of the deceased).
"""

IdentifiedThemes = extractor.discover_themes(trim_approach="truncate",
                          extra_instructions=extra_instructions)

print(extractor.identified_themes)

{
  "family_input": "Lack of sufficient involvement, consultation, or communication with family members or significant others in risk assessments, care planning, and treatment decisions.",
  "family_awareness": "Failure to adequately inform family or carers about patient status, discharge, risk levels, or changes in care plans.",
  "family_support": "Insufficient support, guidance, or resources provided to family members or carers to help them assist the patient effectively.",
  "family_concerns": "Failure to take seriously or act upon concerns, observations, or information provided by family or carers about the patient’s condition or risk.",
  "family_communication": "Poor or inconsistent communication channels between healthcare providers and family, including lack of timely updates or feedback.",
  "family_confidentiality": "Overemphasis on patient confidentiality leading to exclusion of family input or withholding critical information from family members.",
  "family_coordination":

In [10]:
themed_reports = extractor.extract_features(
                              feature_model=IdentifiedThemes,
                              force_assign=True,
                              allow_multiple=True)

extractor.tabulate()

Extracting features: 100%|██████████| 237/237 [00:12<00:00, 19.25it/s]


,Category,Count,Percentage
0,family_input,148,62.447257
1,family_awareness,153,64.556962
2,family_support,67,28.270042
3,family_concerns,112,47.257384
4,family_communication,156,65.822785
5,family_confidentiality,31,13.080169
6,family_coordination,112,47.257384
7,family_training,32,13.502110


In [11]:
themed_reports

,url,id,date,coroner,area,receiver,investigation,circumstances,concerns,family_input,family_awareness,family_support,family_concerns,family_communication,family_confidentiality,family_coordination,family_training
5,https://www.judiciary.uk/prevention-of-future-...,2025-0587,2025-11-17,M. Whittle,Other,"Chief Constable, South Yorkshire Police",On 8 March 2024 an investigation was commenced...,Andrew Herrin Dodds was assessed by mental hea...,(1) Police did not pass over relevant details ...,False,True,False,False,True,False,True,False
16,https://www.judiciary.uk/prevention-of-future-...,2025-0571,2025-11-10,J. Turner,"West Sussex, Brighton and Hove","Chief Executive, NHS England",On 23 January 2025 I commenced an investigatio...,"The day before her death, Joanna attended clif...",Whilst I am keenly aware that it is not for co...,True,True,False,True,True,True,True,False
43,https://www.judiciary.uk/prevention-of-future-...,2025-0556,2025-10-24,S. Horstead,Essex,CEO Essex Partnership University NHS Foundatio...,On 5 January 2022 I commenced an investigation...,On a background of diagnoses of severe (treatm...,1. There was a failure on the part of EPUT nur...,True,True,False,False,True,False,True,True
72,https://www.judiciary.uk/prevention-of-future-...,2025-0509,2025-10-13,N. Walker,"Hampshire, Portsmouth and Southampton",Hampshire and Isle of Wight Healthcare (HIOWH)...,On 13 November 2024 I commenced an investigati...,Abigail was a 34-year-old mother of two who su...,Community Mental Health Teams do not receive m...,True,True,False,True,True,False,False,True
95,https://www.judiciary.uk/prevention-of-future-...,2025-0522,2025-09-29,R. Simpson,"Hampshire, Portsmouth and Southampton",CEO - Hampshire and Isle of Wight Healthcare N...,On 16 September 2024 I commenced an investigat...,In July 2023 Naomi contracted viral meningitis...,1. I am concerned that Naomi was never seen fa...,True,False,False,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5952,https://www.judiciary.uk/prevention-of-future-...,2013-0269,2013-10-21,A. Tweddle,County Durham and Darlington,"Chief Constable, Durham Constabulary",On 10 June 2013 I commenced an investigation i...,The deceased had been a firearms and shotgun c...,(1) As part of the process leading to the deci...,False,True,False,False,True,False,False,False
5980,https://www.judiciary.uk/prevention-of-future-...,2013-0243,2013-09-24,A. Tweddle,County Durham and Darlington,"Tees, Esk and Wear Valley NHS Foundation Trust",On 21 February 2013 I commenced an investigati...,The deceased had been admitted to Lanchester R...,(1) The deceased was discharged from hospital ...,True,True,True,True,True,False,False,False
5999,https://www.judiciary.uk/prevention-of-future-...,2013-0227,2013-09-09,P. Harding,Kent Mid and Medway,Kent and Medway NHS and Social Care Partnershi...,On 23 May 2012 I commenced an investigation in...,On the 29 March 2012 Ricky Anderson was inform...,(1) It was established in evidence at the inqu...,False,False,False,True,True,False,True,False
6014,https://www.judiciary.uk/prevention-of-future-...,2013-0213,2013-08-21,C. Wilkinson,"West Sussex, Brighton and Hove","Lisa Rodrigues, Chief Executive, Sussex Partne...",On 6 June 2012 the senior coroner for West Sus...,Mr Walker had a history of mental health issue...,"(1) The consideration of, contribution to and ...",True,True,False,True,True,False,True,False


In [12]:
themed_reports.to_csv('../data/suicide_sig_themed.csv')